<a href="https://colab.research.google.com/github/jainnipun/MachineLearning/blob/master/TextAnalytics/TopicModelling..ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

In [0]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=1, random_state =1, remove=('headers','footers','quotes'))
documents = dataset.data
#len(documents)
#datasets.taget_names

In [0]:
news_df = pd.DataFrame({'document':documents})

# removing everything except alphabets`
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# removing short words
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make all text lowercase
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())


In [0]:
news_df = pd.DataFrame({'documents':documents})

#Removing everything except columns
news_df['clean_doc'] = news_df["documents"].str.replace("[^a-zA-Z#]"," ");

news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x:' '.join([w for w in x.split() if len(w)>3]))

news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x:x.lower())

news_df['clean_doc']


In [0]:
news_df['clean_doc']

In [0]:
import nltk
nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [0]:
#tokenization
tokenized_doc = news_df['clean_doc'].apply(lambda x:x.split())

In [0]:
tokenized_doc = tokenized_doc.apply(lambda x: [w for w in x if w not in stop_words])

In [0]:
#detokenization
detokenized_doc = []
for i in range(len(tokenized_doc)): # or len(news_df['clean_doc'])
  detokenized = ' '.join(tokenized_doc[i])
  detokenized_doc.append(detokenized)

news_df['clean_doc'] = detokenized_doc

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # check shape of the document-term matrix


In [0]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)


In [0]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")


In [0]:
import umap

X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 
c = dataset.target,
s = 10, # size
edgecolor='none'
)
plt.show()
